# GORi: an efficient algorithm to annotate pools of genesets simultaneously, with multiple knowledge bases.

In this notebook, we present GORi: an algorithm developed to conduct enrichment analyses on a pool of genesets, with multiple knowledge bases.

GORi is computationally efficient, and is able to leverage hierarchical relationships between genesets (e.g. marker genes of a cell population and marker genes of its sub-populations) to improve the quality of its enrichment analysis. 

### Installing GORi

GORi can be installed as a Python3 package using pip3 (cf. `README.md`). Following its installation, you can import it with:

In [ ]:
import gori

### Overview of GORi

GORi loads a pool of **priors** associating genes and concepts (*e.g.* the Gene Ontology). It leverages the gene annotations of each prior to identify strong associations between pairs of concepts. 

A prior is a collection of three Python3 dictionaries:
    
- **annotations**: a dict associating gene symbols or uniprot ids (keys) to a set of semantic ids (values).
- **hierarchy**: a dict associating semantic ids (keys) to a set of parents in the hierarchy (values).
- **translations**: a dict associating semantic ids (keys) to their human-readable label (values).

To demonstrate how to use GORi, we will apply it on the results of a scRNA-seq clustering analysis conducted with the fEVE framework (cf. https://github.com/yanisaspic/fEVE) downloaded from an online repository.

*This analysis was conducted on a human glioblastoma dataset, and 10 populations of cells were predicted: C, C.1, C.2, C.3, C.L, C.L.1, C.L.L, C.L.L.1, C.L.L.2 and C.L.L.L. These populations were predicted across multiple resolutions, as indicated by their label (*e.g.* C.L.1 and C.L.L are two sub-populations of C.L). For each population, a set of marker genes was predicted, and all of these informations are stored in the data file `Darmanis_HumGBM.xlsx` of the GORi package.*

##### Setting-up GORi

Using the function `load_feve()`, we will load the results of this analysis as a prior for GORi:

In [ ]:
from gori.loaders import load_feve
from urllib.request import urlretrieve

url = "https://github.com/yanisaspic/GORi/raw/refs/heads/main/data/Darmanis_HumGBM.xlsx"
path = "./Darmanis_HumGBM.xlxs"
urlretrieve(url, path)

feve_prior = load_feve(path=path)

For this analysis, our other priors will correspond to curated knowledge bases associating genes to specific concepts. Eight different priors are readily implemented in GORi:

- **BIOP: biological processes**, from the Gene Ontology (GO).
- **CELC: cellular components**, from the Gene Ontology (GO).
- **CTYP: cell types**, from the Cell Ontology (CL) and CellMarker 2.0.
- **CTYP2: cell types**, from the Cell Ontology (CL) and CellTaxonomy. 
- **DISE: diseases**, from the Comparative Toxicogenomics Database (CTD) and the Medical Subject Headings (MeSH).
- **GENG: gene groups**, from the HUGO Gene Nomenclature Committee (HGNC).
- **MOLF: molecular functions**, from the Gene Ontology (GO).
- **PATH: pathways**, from Reactome.
- **PHEN: phenotypes**, from the Human Phenotype Ontology (HPO).

In order to load them, the function `load_priors()` must be called.

**Note 1:** the priors CTYP, DISE, GENG and PATH must be intialized with `download_priors()` and `setup_priors()` before loading.

**Note 2:** only human protein-coding genes (with UniProtIDs) are handled by these priors.

In [ ]:
from gori.init import download_priors, setup_priors
from gori.loaders import load_priors

priors = {"BIOP", "CELC", "CTYP", "CTYP2", "DISE", "GENG", "MOLF", "PATH", "PHEN"}
priors_requiring_init = {"CTYP", "CTYP2", "DISE", "GENG", "PATH"}

download_priors(priors_requiring_init)
setup_priors(priors_requiring_init)
data = load_priors(priors)

**Note 1:** downloading and setting-up CTYP, CTYP2, DISE, GENG and PATH can be time-consuming. You only need to do it once, as local files will be stored on your machine afterwards (by default, in the folders `./.priors` and `./priors`).

**Note 2:** the first time you load the priors BIOP, CELC, MOLF and PHEN, cache files are automatically saved in your machine (from the `pypath` package). This process can also be time-consuming. If errors are raised after loading one of these priors, they will likely be caused by cache issues. They can be solved by clearing the cache (cf. `README.md`), and regenerating it with `load_priors()`.

It is also possible to load a prior from a collection of three local .json files with the function `load_local()`. For instance, a prior labeled POPU can be loaded if three .json files are saved locally: `POPU_a.json`, `POPU_h.json` and `POPU_t.json` (for annotations, hierarchy and translations, respectively).

We demonstrate how to use this function by loading the DISE prior from the local collection of .json files generated after its initialization:

In [ ]:
from gori.loaders import load_local

dise_prior = load_local(prior="DISE", path="./priors")

**Note:** you should make sure that any prior loaded with GORi is a directed acyclic graph (DAG), rooted to a unique concept (*e.g.* `biological_process`). Otherwise, the analysis might not run smoothly.

##### Running GORi

After loading all of our priors of interest, we will merge them to a single variable, and start the GORi analysis with the function `gori()`. This function expects four arguments:
- **geneset** is a set of gene symbols or Uniprot ids used to conduct the enrichment analysis. We'll use the marker genes predicted with the fEVE analysis.
- **antecedent_prior** is a label indicating which prior should be annotated. We'll annotate the clusters predicted with the fEVE analysis.
- **consequent_prior** is a set of labels indicating which priors should be used to annotate. We'll use the eight default priors available with GORi.
- **data** is a collection of priors. We have loaded them earlier.

In [ ]:
from gori.run import gori

data["FEVE"] = feve_prior
geneset = data["FEVE"]["annotations"].keys()
results = gori(geneset=geneset, antecedent_prior="FEVE", consequent_priors=priors, data=data, save=False)
print(results)

Briefly, the results of a GORi analysis is a collection of four data frames:
- `annotations_counter` reports the number of annotation identified for each gene, with each prior.
- `associations_counter` reports the number of associations identified by GORi at each step of the algorithm.
- `associations` reports the pairwise associations between genesets and terms identified by GORi.
- `words` reports the importance of each individual word to characterize genesets.

By default, these results are saved to a collection of spreadsheets `GORi.xlsx`. An HTML report `GORi.html` is also generated to explore them interactively. Here, because we have set `save=False` when we have run `gori()`, neither documents were generated. 

A more extensive presentation of GORi's results is directly available in the report generated post-analysis.

##### Running GORi with fEVE

The results of a fEVE clustering analysis can be analyzed with GORi directly with the function `gorilon()`. In this case, the fEVE prior is automatically generated, and is annotated with 6 other priors: BIOP, CELC, CTYP, GENG, MOLF and PATH.

In [ ]:
from gori.run import gorilon

results = gorilon(path)
print(results)

This time, we have not set `save=False`, and both the collection of spreadsheets and the HTML report have been generated.

**We recommend users to check the generated report before proceeding with GORi.**

##### Parameterizing GORi

A set of default parameters is used to run a GORi analysis. These parameters are generated with the function `get_parameters()`, and they include:

- **n_genes_threshold:** the minimum number of co-annotated genes required to associate two concepts. Defaults to `5`.
- **pvalue_threshold:** the p-value threshold used to identify strong associations (using Fisher's exact test). Defaults to `0.05`.
- **use_heuristic:** a boolean indicating if GORi's heuristic strategy should be employed to identify strong associations. Defaults to `True`.
- **use_gene_symbol:** a boolean indicating if gene symbols (True) or UniProtIDs (False) should be used. Defaults to `True`.
- **sheets_path:** a path to store the spreadsheets of the GORi analysis. Defaults to `./GORi.xlsx`.
- **report_path:** a path to store the HTML report of the GORi analysis. Defaults to `./GORi.html`.
- **stopwords:** a set of words that should be filtered out from the words overview.
- **wrappers:** wrapper functions required to run GORi.

These settings can be changed prior to the GORi analysis, and must be input to the argument `params` of `gori()` or `gorilon()`:

In [ ]:
from gori.params import get_parameters

params = get_parameters()
params["n_genes_threshold"] = 10
params["pvalue_threshold"] = 0.001
params["use_heuristic"] = False
params["use_gene_symbol"] = False
params["stopwords"] = params["stopwords"] | {"bind, binding"}
results = gori(geneset=geneset, antecedent_prior="FEVE", consequent_priors={"CTYP", "BIOP", "PATH"}, data=data, params=params, save=False)
print(results)